<a href="https://colab.research.google.com/github/carlos-alves-one/-BDA-CW1/blob/main/weather_data_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Goldsmiths University of London
### MSc. Data Science and Artificial Intelligence
### Module: Big Data Analysis
### Author: Carlos Manuel De Oliveira Alves
### Student: cdeol003
### Coursework Project

# Define Goal and Tasks

THE GOAL OF THIS PROJECT IS TO ANALYZE WEATHER DATA FROM A CSV FILE

We will outline the pseudo-code for mapper and reducer functions for each requested task using
the MapReduce computational model on a Hadoop cluster to address the tasks.
We will then provide Python implementations of these pseudo codes.

The tasks are:

1. Finding the difference between the maximum and minimum wind speed for each day.

2. Finding the daily minimum relative humidity.

3. Calculating the daily mean and variance of the dew point temperature.

4. Generating a correlation matrix for the month among relative humidity, wind speed,
   and dry bulb temperature.

# Question No.1

> Find the description statistics for temperature of each day of a given month for the year 2007

# Pseudo-code for the mapper function for Task No.1


---



In [3]:
"""
function mapper1(key, value):
      parse value to get the date and wind speed
      emit (date, wind speed) as the key-value pair
"""


'\nfunction mapper1(key, value):\n      parse value to get the date and wind speed\n      emit (date, wind speed) as the key-value pair\n'


- **`function mapper1(key, value):`**
  - This line defines a function named `mapper1`. The function takes two parameters: `key` and `value`. In the context of MapReduce, each input is typically a key-value pair. The `key` might represent some identifier (often unused in the map step), and the `value` represents the data to be processed.

- **`parse value to get the date and wind speed`**
  - This instruction indicates that the function will process the `value` to extract two specific pieces of information: the date and the wind speed. The parsing method depends on the format of the input data. For example, if the value is a string in the format "YYYY-MM-DD, wind_speed", parsing would involve splitting the string by a delimiter (like a comma) and extracting the relevant parts.

- **`emit (date, wind speed) as the key-value pair`**
  - After parsing the value to extract the date and wind speed, the function "emits" or outputs a new key-value pair. In this context, the new key is the date, and the new value is the wind speed. The emit operation is a fundamental part of the MapReduce model, where each mapper function outputs zero or more key-value pairs, which are then processed by reducer functions.

The purpose of this mapper function is to transform raw data into a format that is more useful for analysis or further processing. By emitting the date and wind speed as key-value pairs, subsequent reducer functions can easily aggregate, summarize, or analyze wind speed data by date. This could be useful in various applications, such as analyzing weather patterns, forecasting, or studying the effects of climate change.